In [4]:
# preprocess image

> Some classical image preprocessing methods are implemented in this script

In [60]:
# | default_exp preprocessing.preprocess_image

In [19]:
# | hide
%load_ext autoreload
%autoreload 2	

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
# | export
import cv2
import numpy as np
import matplotlib.pyplot as plt
from fastcore.all import *
import os
from tqdm.auto import tqdm
from typing import List, Tuple
from pathlib import Path

In [23]:
#| export
import matplotlib as mpl

In [40]:
# | export
from cv_tools.core import *

In [33]:
data_path = Path(os.getenv("DATA_PATH"))
path = Path(data_path, "crack_detect_usm/test_data_sig1_cut_roi")
good_path = Path(data_path, "crack_detect_usm/good_images_cut_roi")

In [34]:
good_images_sig1 = good_path.filter_("sig_1")
crack_images = path.filter_("sig_1")

In [35]:
len(good_images_sig1), len(crack_images)

(19106, 27)

# Create random 5000 roi images as good

In [15]:
np.random.choice?

Docstring:
choice(a, size=None, replace=True, p=None)

Generates a random sample from a given 1-D array

.. versionadded:: 1.7.0

.. note::
    New code should use the `~numpy.random.Generator.choice`
    method of a `~numpy.random.Generator` instance instead;
    please see the :ref:`random-quick-start`.

Parameters
----------
a : 1-D array-like or int
    If an ndarray, a random sample is generated from its elements.
    If an int, the random sample is generated as if it were ``np.arange(a)``
size : int or tuple of ints, optional
    Output shape.  If the given shape is, e.g., ``(m, n, k)``, then
    ``m * n * k`` samples are drawn.  Default is None, in which case a
    single value is returned.
replace : boolean, optional
    Whether the sample is with or without replacement. Default is True,
    meaning that a value of ``a`` can be selected multiple times.
p : 1-D array-like, optional
    The probabilities associated with each entry in a.
    If not given, the sample assumes a unif

In [16]:
r_images = np.random.choice(
good_images_sig1, 5000,replace=False)

In [18]:
symlink_path = Path(data_path, "crack_detect_usm/random_good_cut_roi")
for i in tqdm(r_images,total=5000):
    des_path = Path(symlink_path, i.name)
    des_path.symlink_to(i)

  0%|          | 0/5000 [00:00<?, ?it/s]

# Now process those images 5000 images

In [12]:
blur_img = cv2.GaussianBlur(sm_img, (5, 5), 0)
new_img = blur_img.copy()


edges = cv2.Canny(blur_img, 50, 150)
cntrs = find_contours_binary(edges)
# cntrs = filter(lambda x: cv2.contourArea(x) > 10, cntrs)
mask = np.zeros_like(sm_img)
for c in cntrs:
    cv2.drawContours(mask, [c], -1, (255), thickness=cv2.FILLED)

In [41]:
#| export
def tmp_part(
    img: np.ndarray,
    tmp_img: np.ndarray,
   ):
   x, y, w, h = get_template_part(img, tmp_img)
   return img[y:y+h, x:x+w]

In [52]:
#| export
def get_roi_frm_tmp(
    tmp_part_img:np.ndarray,
    h_idx0:int=0,
    h_idx1:int=350,
    w_idx0:int=190,
    w_idx1:int=-20,
    ):
    return tmp_part_img[h_idx0:h_idx1, w_idx0:w_idx1]

In [53]:
#| export
def apply_filter_(
    img:np.ndarray,
    filters_:List[str],
    ):
    return filter_img_(img, filters_)

In [54]:
#| export
def frm_img_to_roi(
    img:np.ndarray,
    tmp_img:np.ndarray,
    filters_:List[str]=["blur", "sharpen"],
    h_idx0:int=0,
    h_idx1:int=350,
    w_idx0:int=190,
    w_idx1:int=-20,
    ):
    """
    from image template part will be extracted and then apply
    the filters and then roi
    
    """
    tmp_part_img = tmp_part(img, tmp_img)
    roi = get_roi_frm_tmp(
        tmp_part_img, 
        h_idx0, 
        h_idx1, 
        w_idx0, 
        w_idx1)
    roi = apply_filter_(roi, filters_)
    return roi

In [57]:
#| hide
import nbdev

In [61]:
#| hide
nbdev.nbdev_export('04_preprocessing.preprocess_image.ipynb')